In [6]:
import torch
from torch import nn
from d2l import torch as d2l

def dropout_layer (X,dropout):   #X为dropout层的输入，dropout为设置的丢弃概率
    assert 0<=dropout<=1        #丢弃概率介于0，1之间
    if dropout == 1:
       return torch.zeros_like(X) #若丢弃概率为1，则X的全部项均被置0
    if dropout == 0:
       return X                   #若丢弃概率为0，不对X作丢弃操作，直接返回X
    mask=(torch.Tensor(X.shape).uniform_(0,1)>dropout).float() #用uniform函数生成0-1间的随机实数，利用”>"，将大于dropout的记为1，小于dropout的记为0，实现丢弃操作
    return mask*X/(1-dropout) #将mask与X相乘实现丢弃操作，并除以(1-dropout)，这里不使用选中X中元素置0的原因是相乘操作相比选中操作更快

In [7]:
#丢弃法测试
X=torch.arange(16,dtype=torch.float32).reshape((2,8))
print(X)
print(dropout_layer (X,0.))  #丢弃率设置为0
print(dropout_layer (X,0.5)) #丢弃率设置为0.5
print(dropout_layer (X,1))   #丢弃率设置为1

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  0.,  4.,  0.,  0., 10.,  0., 14.],
        [16., 18., 20.,  0.,  0., 26.,  0., 30.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [24]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256
dropout1, dropout2 = 0.2, 0.5
class Net(nn.Module):
    def _init_(self,num_inputs,num_outputs,num_hiddens1,num_hiddens2,is_training=True):
       super(Net,self)._init_()
       self.num_inputs=num_inputs
       self.training=is_training
       self.lin1=nn.Linear(num_inputs,num_hiddens1)
       self.lin2=nn.Linear(num_hiddens1,num_hiddens2)
       self.lin2=nn.Linear(num_hiddens2,num_outputs)
       self.relu=nn.ReLU()
    def forward(self,X):
       H1=self.relu(self.lin1(X.reshape((-1,self.num_inputs))))
       if self.training == True:  #丢弃法仅在训练中使用
           H1=dropout_layer(H1,dropout1)
       H2=self.relu(self.lin2(H1))
       if self.training == True: #丢弃法仅在训练中使用
           H2=dropout_layer(H2,dropout2)
       out=self.lin3(H2)  #output层不再使用丢弃法
       return out
    
net = Net(num_inputs,num_outputs,num_hiddens1,num_hiddens2)

TypeError: __init__() takes 1 positional argument but 5 were given